# Muffin vs Chihuahua Classification (Custom Dataset)

This section demonstrates binary image classification using a custom muffin vs chihuahua dataset and a CNN architecture.

This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [71]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [72]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [73]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.

train_1 = r"C:\Users\LENOVO\Downloads\train_split"
test_1 = r"C:\Users\LENOVO\Downloads\test_split"


In [74]:
# OPTIONAL: Split dataset into train/test folders (run once)
import shutil
from pathlib import Path
import random

def split_dataset(source_dir, train_ratio=0.8):
    """
    Split dataset into train and test directories
    source_dir: Path to folder containing class subfolders (chihuahua, muffin)
    train_ratio: Percentage of data for training (0.8 = 80%)
    """
    source = Path(source_dir)
    parent = source.parent
    
    # Create train and validation directories (avoid naming conflict)
    train_dir = parent / "train_split"
    test_dir = parent / "test_split"
    
    # Get all class folders (chihuahua, muffin)
    for class_folder in source.iterdir():
        if class_folder.is_dir():
            class_name = class_folder.name
            
            # Create class folders in train and test
            (train_dir / class_name).mkdir(parents=True, exist_ok=True)
            (test_dir / class_name).mkdir(parents=True, exist_ok=True)
            
            # Get all images in this class
            images = list(class_folder.glob("*.*"))
            random.shuffle(images)
            
            # Split images
            split_idx = int(len(images) * train_ratio)
            train_images = images[:split_idx]
            test_images = images[split_idx:]
            
            # Copy images to train folder
            for img in train_images:
                shutil.copy2(img, train_dir / class_name / img.name)
            
            # Copy images to test folder
            for img in test_images:
                shutil.copy2(img, test_dir / class_name / img.name)
            
            print(f"{class_name}: {len(train_images)} train, {len(test_images)} test")
    
    print(f"\nDataset split complete!")
    print(f"Train directory: {train_dir}")
    print(f"Test directory: {test_dir}")

# Remove the hash and run once to split the dataset
# split_dataset(r"C:\Users\LENOVO\Downloads\test", train_ratio=0.8)

In [75]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [76]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_1,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_1,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_1,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 758 images belonging to 3 classes.
Found 189 images belonging to 3 classes.
Found 189 images belonging to 3 classes.
Found 237 images belonging to 3 classes.
Found 237 images belonging to 3 classes.


In [77]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION AND DROPOUT

from tensorflow.keras import regularizers



initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)



model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [78]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [79]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.5343 - loss: -12116.8184 - val_accuracy: 0.5397 - val_loss: -70929.8281
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.5343 - loss: -12116.8184 - val_accuracy: 0.5397 - val_loss: -70929.8281
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5409 - loss: -898319.1875 - val_accuracy: 0.5397 - val_loss: -3183275.7500
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5409 - loss: -898319.1875 - val_accuracy: 0.5397 - val_loss: -3183275.7500
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5409 - loss: -14285393.0000 - val_accuracy: 0.5397 - val_loss: -38399440.0000
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5409 - loss: -14285393.0000 - val_accuracy: 0.5397 - val_loss: -38399440.0000
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5409 - loss: -109683632.0000 - val_accuracy: 0.5397 - val_loss: -241253744.0000
Epoch 5/10
24/24 ━━━━━

In [80]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5401 - loss: -72706793472.0000
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5401 - loss: -72706793472.0000
Test Accuracy: 0.5400843620300293
Test Accuracy: 0.5400843620300293


In [81]:
# SAVE THE IMPROVED MODEL
model.save('exercise_6_trained_model_improved.h5')

In [82]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [83]:
# Example usage - Run 1 and 2 prediction and confidence:
predict_image(r"C:\Users\LENOVO\Downloads\test_split\chihuahua\img_0_1082.jpg", model_path='exercise_6_trained_model_improved.h5')
predict_image(r"C:\Users\LENOVO\Downloads\test_split\muffin\img_0_423.jpg", model_path='exercise_6_trained_model_improved.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Prediction: Chihuahua (confidence: 1.00)
Prediction: Chihuahua (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Prediction: Chihuahua (confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Prediction: Chihuahua (confidence: 1.00)


# Cats vs Dogs Classification (Kaggle)

This section demonstrates how to use the Kaggle Cats vs Dogs dataset with the improved CNN architecture.

In [84]:
# DATASET DIRECTORY CONFIGURATION (Cats vs Dogs)
catsdogs_train_dir = r"C:\Users\LENOVO\Downloads\cats_and_dogs\training_set\training_set"
catsdogs_test_dir = r"C:\Users\LENOVO\Downloads\cats_and_dogs\test_set\test_set"

In [85]:
# DATA PREPROCESSING & AUGMENTATION (Cats vs Dogs)
catsdogs_img_size = (128, 128)
catsdogs_batch_size = 32
catsdogs_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
catsdogs_test_datagen = ImageDataGenerator(rescale=1./255)

catsdogs_train_generator = catsdogs_train_datagen.flow_from_directory(
    catsdogs_train_dir,
    target_size=catsdogs_img_size,
    batch_size=catsdogs_batch_size,
    class_mode='binary',
    subset='training'
)
catsdogs_val_generator = catsdogs_train_datagen.flow_from_directory(
    catsdogs_train_dir,
    target_size=catsdogs_img_size,
    batch_size=catsdogs_batch_size,
    class_mode='binary',
    subset='validation'
)
catsdogs_test_generator = catsdogs_test_datagen.flow_from_directory(
    catsdogs_test_dir,
    target_size=catsdogs_img_size,
    batch_size=catsdogs_batch_size,
    class_mode='binary',
    shuffle=False
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [86]:
# IMPROVED CNN MODEL FOR CATS VS DOGS

from tensorflow.keras import regularizers

initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


catsdogs_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(catsdogs_img_size[0], catsdogs_img_size[1], 3),
                kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


catsdogs_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [87]:
# TRAINING THE CNN (Cats vs Dogs)
catsdogs_history = catsdogs_model.fit(
    catsdogs_train_generator,
    epochs=10,
    validation_data=catsdogs_val_generator
)

# EVALUATE THE MODEL (Cats vs Dogs)
catsdogs_test_loss, catsdogs_test_acc = catsdogs_model.evaluate(catsdogs_test_generator)
print(f"Cats vs Dogs Test Accuracy: {catsdogs_test_acc}")

# SAVE THE IMPROVED MODEL (Cats vs Dogs)
catsdogs_model.save('exercise_6_custom_danocup.h5')

# SIMPLE INFERENCE SCRIPT (Cats vs Dogs)
def catsdogs_predict_image(img_path, model_path='exercise_6_custom_danocup.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=catsdogs_img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Dog" if pred >= 0.5 else "Cat"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

# Example usage - Replace with actual image paths:
# catsdogs_predict_image(r"C:\Users\LENOVO\Downloads\cats_and_dogs\test_set\test_set\cats\cat.4001.jpg")
# catsdogs_predict_image(r"C:\Users\LENOVO\Downloads\cats_and_dogs\test_set\test_set\dogs\dog.4001.jpg")

Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 246s 1s/step - accuracy: 0.5094 - loss: 0.8807 - val_accuracy: 0.4997 - val_loss: 0.7773
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 246s 1s/step - accuracy: 0.5094 - loss: 0.8807 - val_accuracy: 0.4997 - val_loss: 0.7773
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 124s 618ms/step - accuracy: 0.5020 - loss: 0.7508 - val_accuracy: 0.5428 - val_loss: 0.7329
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 124s 618ms/step - accuracy: 0.5020 - loss: 0.7508 - val_accuracy: 0.5428 - val_loss: 0.7329
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 123s 614ms/step - accuracy: 0.5217 - loss: 0.7219 - val_accuracy: 0.5147 - val_loss: 0.7170
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 123s 614ms/step - accuracy: 0.5217 - loss: 0.7219 - val_accuracy: 0.5147 - val_loss: 0.7170
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 356s 2s/step - accuracy: 0.5231 - loss: 0.7119 - val_accuracy: 0.5628 - val_loss: 0.7020
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 356s 2s/step - accuracy: 0.5231 - loss: 0.711

Cats vs Dogs Test Accuracy: 0.5996045470237732


Muffin vs Chihuahua - ResNet CNN Architecture

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

In [2]:
# model
MODEL_PATH = 'muffin_vs_chihuahua_model.h5'

# test folder
PREDICT_DIR = 'predict' 

IMG_SIZE = (224, 224)

In [3]:
print(f"Loading model from {MODEL_PATH}...")
try:
    model = load_model(MODEL_PATH)
    print("Model loaded successfully.")
except Exception as e:
    print(f"\n[ERROR] Could not load model: {e}")
    raise

Loading model from muffin_vs_chihuahua_model.h5...


Model loaded successfully.


In [4]:
# 'chihuahua' is Class 0
# 'muffin' is Class 1
CLASS_NAMES = ['Chihuahua', 'Muffin']

In [5]:
print(f"\n--- Starting Predictions in '{PREDICT_DIR}' ---")

try:
    image_files = [f for f in os.listdir(PREDICT_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if not image_files:
        print(f"[WARNING] No images found in '{PREDICT_DIR}'.")
        
except FileNotFoundError:
    print(f"\n[ERROR] The directory '{PREDICT_DIR}' was not found.")
    print(f"Please create a folder named 'predict' in your project directory.")
    raise

for file_name in image_files:
    img_path = os.path.join(PREDICT_DIR, file_name)

    try:
        
        img = image.load_img(img_path, target_size=IMG_SIZE)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
    
        img_array = img_array / 255.0 
        
        prediction = model.predict(img_array)
        
        raw_score = prediction[0][0]

        # result
        if raw_score > 0.5:
            result_class = CLASS_NAMES[1] # 'Muffin'
            confidence = raw_score * 100
            raw_score_to_print = 1 - raw_score 
        else:
            result_class = CLASS_NAMES[0] # 'Chihuahua'
            confidence = (1 - raw_score) * 100
            raw_score_to_print = raw_score 

        print(f"\n[PREDICTION] {img_path}")
        print(f"  >> Result: {result_class}")
        print(f"  >> Confidence: {confidence:.2f}%")
        
        print(f"  >> Raw score: {raw_score_to_print:.4f}") 

    except Exception as e:
        print(f"\n[ERROR] Could not process image {img_path}: {e}")

print("\nAll predictions completed.")


--- Starting Predictions in 'predict' ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

[PREDICTION] predict\c1.jpg
  >> Result: Chihuahua
  >> Confidence: 81.91%
  >> Raw score: 0.1809
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step

[PREDICTION] predict\c2.jpg
  >> Result: Chihuahua
  >> Confidence: 70.98%
  >> Raw score: 0.2902
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step

[PREDICTION] predict\m1.jpg
  >> Result: Muffin
  >> Confidence: 85.01%
  >> Raw score: 0.1499
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step

[PREDICTION] predict\m2.jpg
  >> Result: Muffin
  >> Confidence: 82.59%
  >> Raw score: 0.1741

All predictions completed.
